# Calendar transformation

Below you can find a `calendar` dataframe  which constitutes the promotion calendar for a number of products.

- `Start date` indicates the first day of a promotion and can be any day of the week,
- `End date` is the last day of a promotion and can also be any day of the week,
- `Product ID` represents the product undergoing the promotion,
- `Price` refers to the final price to the customer, and
- `Org price` is the original product of the price without the promotion (MSRP).

Please write the code that you would use to change this calendar to a weekly calendar of products, with Saturday as the last day of the week as shown in below. Should part of the week be on promotion, the price should be the average price in this week.

In [1]:
import pandas as pd

calendar = pd.DataFrame({
    'start_Date': ['01/01/2019', '01/01/2019', '02/15/2019'],
    'end_Date': ['01/16/2019', '01/21/2019', '02/20/2019'],
    'product_ID': ['A', 'B', 'A'],
    'price': [100, 150, 150],
    'org_Price': [200, 300, 200]
})
calendar = pd.DataFrame({
    'start_Date': ['01/01/2019', '01/01/2019', '02/15/2019'],
    'end_Date': ['01/16/2019', '01/21/2019', '02/20/2019'],
    'product_ID': ['A', 'B', 'A'],
    'price': [100, 150, 150],
    'org_Price': [200, 300, 200]
})



## Expected output

![](result.png)

## Solution



<details>
  <summary>Click to view the code</summary>

```python
    ww = []
    # make one dayly calendar for each product
    for p, product_cal in calendar.groupby('product_ID'):
        # make a dayly calendar
        weekly = pd.date_range(
            "12/30/2018",  # numerically match the example
            "02/23/2019"
        ).to_frame().rename({0: 'date'}, axis=1)
        # add names of the week and keep track of product
        weekly['dayname']= weekly.date.dt.day_name()
        weekly['product_ID'] = p
        # register price as price on promotion or original price
        for _, row in product_cal.iterrows():
            weekly.loc[:, 'org_Price'] = row.org_Price
            weekly.loc[row.start_Date:row.end_Date, 'price'] = row.price
            weekly.loc[weekly.price.isna(), 'price'] = row.org_Price
        # construct week numbers for grouping
        prev = weekly.index[0]
        wn=0
        sunday_ind = weekly[weekly.dayname == 'Sunday'].index.tolist()
        sunday_ind.append(weekly.date.max())  # add last day
        for sun in sunday_ind:
            weekly.loc[prev:sun, 'weeknumber'] = wn
            prev=sun
            wn+=1
        # group and calculate week's average price
        weekly['avg_price']= weekly.groupby('weeknumber').price.transform("mean")
        ww.append(weekly.copy())
    # join calendars for both products
    final = pd.concat(ww)
    # show Saturdays
    final[final.dayname == 'Saturday'].drop(
        ['price', 'weeknumber'], axis=1
    ).rename(
        {'avg_price': 'price'}, axis=1
    )
```
</details>
